In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Loading the dataset
dataset = pd.read_csv('train.csv')

In [ ]:
# Display the dataset
print(dataset)

In [ ]:
import re

# Emoji Cleaner Regex
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002500-\U00002BEF"  # chinese char
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"  # dingbats
                           u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Turkish or Azerbaijani Character Regex
turkish_or_azerbaijani_pattern = re.compile("[a-zA-ZğüşıöçƏəĞÜŞİÖÇ]")


# Clear emojis and make all lowercased
dataset['content'] = dataset['content'].apply(lambda x: remove_emojis(str(x)).lower())
# Filter rows where 'content' column contains Turkish or Azerbaijani characters, 
dataset = dataset[dataset['content'].apply(lambda x: bool(turkish_or_azerbaijani_pattern.search(str(x))))]


# Reset index after filtering
dataset.reset_index(drop=True, inplace=True)



In [ ]:
# Seperating contents, scores and upvotes
contents = dataset['content'].values.tolist()
scores = dataset['score'].values.tolist()
upvotes = dataset['upvotes'].values.tolist()

In [ ]:
# Split the dataset into training and testing sets
split_point = int(len(contents) * 0.8)

train_contents, test_contents = contents[:split_point], contents[split_point:]
train_scores, test_scores = scores[:split_point], scores[split_point:]
train_upvotes, test_upvotes = upvotes[:split_point], upvotes[split_point:]

In [ ]:
contents